Check Elasticsearch is running

In [3]:
import requests

requests.get('http://localhost:9200/_cluster/health').json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

In [10]:
print(requests.get('http://localhost:9200/_cat/indices').text)

yellow open squad_docs WIZ3Wta7QU2KHa1mUYnwUw 1 1 1204 0 18.4mb 18.4mb
yellow open label      HcGaYu5QRk2J-Oa3i2p39w 1 1    0 0   208b   208b



Connect to the new index using Haystack Elasticsearch doc store

In [7]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

We also create new index called `aurelius`:

In [11]:
doc_store = ElasticsearchDocumentStore(host='localhost',
                                       username='', password='',
                                       index='aurelius')

04/10/2021 13:43:53 - INFO - elasticsearch -   PUT http://localhost:9200/aurelius [status:200 request:0.200s]
04/10/2021 13:43:53 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.001s]


Format data into correct format for Elastic, which we will index using Haystack. We format the data using list of dict objects, each with format:

```json
{
    'text': '<paragraph here>',
    'meta': {
        'source': "<book name, eg 'Meditations'>"
    }
}
```

We need to load the data first, then reformat:

In [15]:
with open('../../data/meditations.txt', 'r') as f:
    data = f.read()

In [17]:
data = data.split('\n')

In [18]:
len(data)

508

In [19]:
data_json = [
    {'text': paragraph,
     'meta': {'book': 'meditations'}}
    for paragraph in data
]

In [20]:
data_json[:3]

[{'text': 'From my grandfather Verus I learned good morals and the government of my temper.',
  'meta': {'book': 'meditations'}},
 {'text': 'From the reputation and remembrance of my father, modesty and a manly character.',
  'meta': {'book': 'meditations'}},
 {'text': 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
  'meta': {'book': 'meditations'}}]

Now we feed into Elasticsearch

In [21]:
doc_store.write_documents(data_json)

04/10/2021 13:51:33 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.872s]
04/10/2021 13:51:34 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.034s]


Query the index to check we have **508** documents:

In [22]:
requests.get('http://localhost:9200/aurelius/_count').json()

{'count': 508,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}